# KEM-DEM framework


## AES encapsulation with RSA - Padding

In [6]:
import os

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


def encapsulate_key_rsa(key, iv, public_key):
    # Notice that we're using OAEP padding here
    ct_key = public_key.encrypt(
        key, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    ct_iv = public_key.encrypt(
        iv, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    return ct_key, ct_iv


def decapsulate_key_rsa(kem_capsule, private_key):
    ct_key, ct_iv = kem_capsule
    key = private_key.decrypt(
        ct_key, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    iv = private_key.decrypt(
        ct_iv, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    return key, iv


def encrypt_message_aes(message, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ct = encryptor.update(message) + encryptor.finalize()
    return ct


def decrypt_message_aes(ciphertext, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    pt = decryptor.update(ciphertext) + decryptor.finalize()
    return pt


# Alice wants to send an encrypted message to Bob

# Alice generates a random AES key and IV
alice_key = os.urandom(32)  # 256-bit key
alice_iv = os.urandom(16)  # 128-bit IV

# Bob generates an RSA key pair
bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())

# Bob's public key is shared with Alice
bob_public_key = bob_private_key.public_key()

# Alice encapsulates the key and IV using Bob's public key
kem_capsule = encapsulate_key_rsa(alice_key, alice_iv, bob_public_key)

# Alice encrypts the message using the AES key and IV
# We need to pad this message to be a multiple of 16 bytes
pt = b"Hello, Bob!" + b"\x00" * (16 - len(b"Hello, Bob!") % 16)
ct = encrypt_message_aes(pt, alice_key, alice_iv)

# Bob decapsulates the key and IV using his private key
bob_key, bob_iv = decapsulate_key_rsa(kem_capsule, bob_private_key)

# Bob decrypts the message using the AES key and IV
pt2 = decrypt_message_aes(ct, bob_key, bob_iv)

assert pt == pt2